In [ ]:
import requests
import pandas as pd
import pyodbc
from io import StringIO
from datetime import datetime

url = f'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol=NVDA&outputsize=compact&datatype=csv&apikey=TS6CPXKQF3EZ5BYM'
r = requests.get(url)

# 2. Ler direto em um DataFrame (sem salvar em disco)
df = pd.read_csv(StringIO(r.text))

hora_atual = datetime.now().time().strftime("%H:%M:%S")

# 3. Tratar dados
df.drop_duplicates(inplace=True)
df['timestamp'] = pd.to_datetime(df['timestamp'])
df['year'] = df['timestamp'].dt.year
df['month'] = df['timestamp'].dt.month
df['day'] = df['timestamp'].dt.day
df['time'] = hora_atual
df['ticket'] = "NVDA"

# Último dia disponível no DataFrame
ultimo_dia = df['timestamp'].max().date()

# Checar se é sábado (5) ou domingo (6)
if ultimo_dia.weekday() < 5:  
    df = df[df['timestamp'].dt.date == ultimo_dia]
    df = df.drop(columns=['timestamp'])
else:
    print("Hoje é fim de semana, não há dados de mercado.")
             
# Conexão
server = "ekp-svr-datafinance.database.windows.net"
database = "BD_Dados_Finance"
username = "admin-db"
password = "Pedrica1009@"
driver = '{ODBC Driver 18 for SQL Server}'  # certifique-se que esse driver está instalado

# Conectar
conn = pyodbc.connect(
    f'DRIVER={driver};SERVER={server};DATABASE={database};UID={username};PWD={password};Encrypt=yes;TrustServerCertificate=no;Connection Timeout=30;'
)

cursor = conn.cursor()

# 5. Inserir no banco
valores = [
    (
        row['ticket'],
        int(row['year']),
        int(row['month']),
        int(row['day']),
        row['time'],
        float(row['open']),
        float(row['high']),
        float(row['low']),
        float(row['close']),
        float(row['volume'])
    )
    for _, row in df.iterrows()
]

sql = """
INSERT INTO NVDA (ticket, year, month, day, time, [open], [high], [low], [close], volume)
VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
"""

cursor.fast_executemany = True
cursor.executemany(sql, valores)
conn.commit()   

# Fechar conexão
cursor.close()
conn.close()
